In [3]:
print('Hello')

Hello



"""
# Нейросетевой подход к решению прямой задачи кинематики робота-манипулятора

Автор: [я]  
Дата: [Дата]

## Цель работы:
Разработка нейросетевой модели для решения прямой кинематической задачи робота-манипулятора 
на основе данных, полученных оптическими или лазерными методами.
"""



In [13]:

# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

# Фреймворки для глубокого обучения
import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras import layers, models, callbacks

# Для работы с 3D графикой и робототехникой
# import pytransform3d.rotations as ptr
# import pytransform3d.transformations as ptt
# import pytransform3d.visualizer as ptv

# Дополнительные библиотеки для обработки данных
# import open3d as o3d  # Для работы с облаками точек (если используются лазерные данные)
# import cv2  # Для обработки изображений (если используются оптические данные)

# Версии библиотек
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)



TensorFlow: 2.16.2
Keras: 3.9.2


# %% [markdown]
"""
## 1. Загрузка и предварительная обработка данных

Данные могут включать:
- Углы сочленений робота (входные данные)
- Положение и ориентацию конечного эффектора (целевые переменные)
- Данные от оптических/лазерных датчиков (дополнительные входные данные)
"""
# %%

In [14]:

# Загрузка данных (пример - замените на свои данные)
# Предполагаем, что данные хранятся в CSV файле
try:
    data = pd.read_csv('robot_kinematic_data.csv')
    print("Данные успешно загружены. Пример данных:")
    display(data.head())
except FileNotFoundError:
    print("Файл с данными не найден. Создаем демонстрационные данные.")
    # Генерация синтетических данных для примера
    num_samples = 1000
    num_joints = 6  # Пример для 6-осевого манипулятора
    
    # Случайные углы в сочленениях
    joint_angles = np.random.uniform(-np.pi, np.pi, size=(num_samples, num_joints))
    
    # Простая прямая кинематика для демонстрации (замените на реальную модель)
    end_effector_pos = np.zeros((num_samples, 3))
    for i in range(num_samples):
        # Простая модель - сумма углов с весами
        end_effector_pos[i, 0] = np.sum(joint_angles[i, :3]) * 0.5
        end_effector_pos[i, 1] = np.sum(joint_angles[i, 3:]) * 0.5
        end_effector_pos[i, 2] = np.mean(joint_angles[i]) * 0.3
    
    data = pd.DataFrame(
        np.column_stack([joint_angles, end_effector_pos]),
        columns=[f'joint_{i}' for i in range(num_joints)] + ['pos_x', 'pos_y', 'pos_z']
    )

# Разделение на признаки и целевую переменную
X = data[[f'joint_{i}' for i in range(num_joints)]].values
y = data[['pos_x', 'pos_y', 'pos_z']].values  # Можно добавить ориентацию

# Нормализация данных
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_scaled, test_size=0.2, random_state=42
)

print(f"Размеры выборок: X_train={X_train.shape}, X_test={X_test.shape}")


Файл с данными не найден. Создаем демонстрационные данные.
Размеры выборок: X_train=(800, 6), X_test=(200, 6)


# %% [markdown]
"""
## 2. Визуализация данных

Проведем анализ распределения данных и их визуализацию.
"""
# %%

In [ ]:


# Визуализация распределения углов в сочленениях
plt.figure(figsize=(12, 6))
for i in range(num_joints):
    sns.histplot(data[f'joint_{i}'], kde=True, label=f'Сочленение {i}')
plt.title('Распределение углов в сочленениях')
plt.xlabel('Угол (рад)')
plt.ylabel('Частота')
plt.legend()
plt.show()

# Визуализация положений конечного эффектора
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(y[:, 0], y[:, 1], y[:, 2], c='r', marker='o', alpha=0.3)
ax.set_title('Пространство положений конечного эффектора')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()
# %%
# %% [markdown]
"""
## 3. Построение нейросетевой модели

Будем исследовать несколько архитектур нейронных сетей для решения задачи.
"""
# %%
def build_baseline_model(input_shape, output_shape):
    """Базовая модель - многослойный перцептрон"""
    model = models.Sequential([
        layers.Dense(64, activation='relu', input_shape=input_shape),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dense(output_shape[0])
    ])
    
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    
    return model

def build_advanced_model(input_shape, output_shape):
    """Усовершенствованная модель с более сложной архитектурой"""
    model = models.Sequential([
        layers.Dense(128, activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(64, activation='relu'),
        layers.Dense(output_shape[0])
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae', 'cosine_proximity']
    )
    
    return model

# Параметры моделей
input_shape = (X_train.shape[1],)
output_shape = (y_train.shape[1],)

# Создаем модели
baseline_model = build_baseline_model(input_shape, output_shape)
advanced_model = build_advanced_model(input_shape, output_shape)

# Выводим архитектуру моделей
print("Базовая модель:")
baseline_model.summary()

print("\nУсовершенствованная модель:")
advanced_model.summary()
# %%
# %% [markdown]
"""
## 4. Обучение моделей
"""
# %%
# Callbacks для обучения
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=10,
    min_lr=1e-6
)

# Обучение базовой модели
print("Обучение базовой модели...")
baseline_history = baseline_model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Обучение усовершенствованной модели
print("\nОбучение усовершенствованной модели...")
advanced_history = advanced_model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)
# %%
# %% [markdown]
"""
## 5. Оценка результатов
"""
# %%
# Функция для визуализации истории обучения
def plot_history(history, title):
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Обучающая')
    plt.plot(history.history['val_loss'], label='Валидационная')
    plt.title(f'{title} - Функция потерь')
    plt.ylabel('MSE')
    plt.xlabel('Эпоха')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='Обучающая')
    plt.plot(history.history['val_mae'], label='Валидационная')
    plt.title(f'{title} - Средняя абсолютная ошибка')
    plt.ylabel('MAE')
    plt.xlabel('Эпоха')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Визуализация истории обучения
plot_history(baseline_history, 'Базовая модель')
plot_history(advanced_history, 'Усовершенствованная модель')

# Оценка на тестовых данных
print("Оценка базовой модели на тестовых данных:")
baseline_test_loss, baseline_test_mae = baseline_model.evaluate(X_test, y_test, verbose=0)
print(f"Test MSE: {baseline_test_loss:.4f}, Test MAE: {baseline_test_mae:.4f}")

print("\nОценка усовершенствованной модели на тестовых данных:")
advanced_test_loss, advanced_test_mae, _ = advanced_model.evaluate(X_test, y_test, verbose=0)
print(f"Test MSE: {advanced_test_loss:.4f}, Test MAE: {advanced_test_mae:.4f}")

# Предсказания на тестовых данных
y_pred_baseline = baseline_model.predict(X_test)
y_pred_advanced = advanced_model.predict(X_test)

# Обратное масштабирование данных
y_test_original = scaler_y.inverse_transform(y_test)
y_pred_baseline_original = scaler_y.inverse_transform(y_pred_baseline)
y_pred_advanced_original = scaler_y.inverse_transform(y_pred_advanced)

# Визуализация предсказаний
fig = plt.figure(figsize=(15, 10))

# Предсказания по X координате
plt.subplot(2, 2, 1)
plt.scatter(y_test_original[:, 0], y_pred_baseline_original[:, 0], alpha=0.3)
plt.plot([y_test_original.min(), y_test_original.max()], 
         [y_test_original.min(), y_test_original.max()], 'k--')
plt.title('Базовая модель - X координата')
plt.xlabel('Истинные значения')
plt.ylabel('Предсказания')

plt.subplot(2, 2, 2)
plt.scatter(y_test_original[:, 0], y_pred_advanced_original[:, 0], alpha=0.3)
plt.plot([y_test_original.min(), y_test_original.max()], 
         [y_test_original.min(), y_test_original.max()], 'k--')
plt.title('Усовершенствованная модель - X координата')
plt.xlabel('Истинные значения')
plt.ylabel('Предсказания')

# Предсказания по Y координате
plt.subplot(2, 2, 3)
plt.scatter(y_test_original[:, 1], y_pred_baseline_original[:, 1], alpha=0.3)
plt.plot([y_test_original.min(), y_test_original.max()], 
         [y_test_original.min(), y_test_original.max()], 'k--')
plt.title('Базовая модель - Y координата')
plt.xlabel('Истинные значения')
plt.ylabel('Предсказания')

plt.subplot(2, 2, 4)
plt.scatter(y_test_original[:, 1], y_pred_advanced_original[:, 1], alpha=0.3)
plt.plot([y_test_original.min(), y_test_original.max()], 
         [y_test_original.min(), y_test_original.max()], 'k--')
plt.title('Усовершенствованная модель - Y координата')
plt.xlabel('Истинные значения')
plt.ylabel('Предсказания')

plt.tight_layout()
plt.show()
# %%
# %% [markdown]
"""
## 6. Интеграция с оптическими/лазерными данными

Дополним модель обработкой данных от внешних датчиков.
"""
# %%
# Пример функции для обработки облака точек (если используются лазерные данные)
def process_point_cloud(point_cloud):
    """Обработка облака точек для извлечения признаков"""
    # Пример: преобразование облака точек в гистограмму расстояний
    if isinstance(point_cloud, o3d.geometry.PointCloud):
        points = np.asarray(point_cloud.points)
    else:
        points = point_cloud
    
    # Вычисление расстояний до центра
    distances = np.linalg.norm(points - np.mean(points, axis=0), axis=1)
    
    # Создание гистограммы
    hist, bins = np.histogram(distances, bins=32, range=(0, np.max(distances)))
    
    return hist

# Пример функции для обработки изображений (если используются оптические данные)
def process_optical_data(image):
    """Извлечение признаков из изображения"""
    # Преобразование в оттенки серого
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Применение SIFT для извлечения признаков
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute(gray, None)
    
    # Усреднение дескрипторов
    if descriptors is not None:
        avg_descriptors = np.mean(descriptors, axis=0)
    else:
        avg_descriptors = np.zeros(128)  # Размерность SIFT дескриптора
    
    return avg_descriptors

# Пример интеграции с нейросетью (дополнительные входные данные)
def build_multimodal_model(joint_input_shape, sensor_input_shape, output_shape):
    """Модель с несколькими входами для обработки разных типов данных"""
    
    # Вход для углов сочленений
    joint_input = layers.Input(shape=joint_input_shape, name='joint_input')
    x = layers.Dense(64, activation='relu')(joint_input)
    x = layers.Dense(128, activation='relu')(x)
    
    # Вход для сенсорных данных
    sensor_input = layers.Input(shape=sensor_input_shape, name='sensor_input')
    y = layers.Dense(128, activation='relu')(sensor_input)
    y = layers.Dense(64, activation='relu')(y)
    
    # Объединение ветвей
    combined = layers.concatenate([x, y])
    
    # Дополнительные слои
    z = layers.Dense(256, activation='relu')(combined)
    z = layers.Dropout(0.3)(z)
    z = layers.Dense(128, activation='relu')(z)
    
    # Выход
    output = layers.Dense(output_shape[0])(z)
    
    # Создание модели
    model = models.Model(
        inputs=[joint_input, sensor_input],
        outputs=output
    )
    
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Пример использования (нужны реальные данные сенсоров)
# multimodal_model = build_multimodal_model(
#     joint_input_shape=(num_joints,),
#     sensor_input_shape=(sensor_feature_size,),
#     output_shape=(y_train.shape[1],)
# )
# %%
# %% [markdown]
"""
## 7. Заключение и выводы

В данной работе была исследована возможность применения нейросетевых подходов 
для решения прямой кинематической задачи робота-манипулятора. Основные результаты:

1. Были реализованы и протестированы несколько архитектур нейронных сетей.
2. Показано, что нейросетевые модели могут эффективно решать задачу прямой кинематики.
3. Исследованы возможности интеграции данных от внешних датчиков (оптических/лазерных).

Перспективные направления дальнейших исследований:
- Использование более сложных архитектур (например, графовые нейросети для моделирования структуры робота)
- Интеграция временных данных (рекуррентные архитектуры)
- Применение методов трансферного обучения для адаптации к разным роботам
"""
# %%
# Сохранение моделей
baseline_model.save('baseline_kinematic_model.h5')
advanced_model.save('advanced_kinematic_model.h5')

print("Модели успешно сохранены.")